<a href="https://colab.research.google.com/github/deividsmek/concurso_itaipu/blob/master/Meta_Vendas_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install dash dash-bootstrap-components pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [13]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from datetime import datetime

# Nome do arquivo CSV onde os dados serão armazenados
csv_file = 'vendas_diarias.csv'

# Meta de vendas para o ano
meta_vendas = 50000

# Inicialização dos dados
dias_do_ano = pd.date_range(start='2025-01-01', end='2025-12-31')

# Função para carregar dados do CSV
def carregar_dados():
    try:
        df = pd.read_csv(csv_file, parse_dates=['Data'])
    except FileNotFoundError:
        df = pd.DataFrame({'Data': dias_do_ano, 'Vendas': [0] * len(dias_do_ano)})
        salvar_dados(df)  # Salvar o arquivo se não existir
    return df

# Função para salvar dados no CSV
def salvar_dados(df):
    df.to_csv(csv_file, index=False)

# Função para inserir vendas diárias
def inserir_vendas(valor):
    hoje = datetime.now().date()
    df = carregar_dados()
    if hoje in df['Data'].dt.date.values:
        df.loc[df['Data'].dt.date == hoje, 'Vendas'] = valor
        salvar_dados(df)
        print(f"Valor de {valor} inserido para o dia {hoje}.")
    else:
        print("Erro: Data fora do intervalo definido.")

# Inicializa a aplicação Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Input(id='input-vendas', type='number', placeholder='Insira o valor de vendas', value=0),
    html.Button(id='submit-button', n_clicks=0, children='Inserir Vendas'),
    dcc.Graph(id='progresso-diario'),
    dcc.Graph(id='progresso-anual')
])

@app.callback(
    [Output('progresso-diario', 'figure'),
     Output('progresso-anual', 'figure')],
    [Input('submit-button', 'n_clicks')],
    [Input('input-vendas', 'value')]
)
def update_graphs(n_clicks, valor_vendas):
    if n_clicks > 0 and valor_vendas is not None:
        inserir_vendas(valor_vendas)

    # Recarregar os dados do CSV
    df = carregar_dados()

    # Gráfico de barras das vendas diárias
    fig_diario = px.bar(df, x='Data', y='Vendas',
                        labels={'Data': 'Dias do Ano', 'Vendas': 'Vendas Diárias'},
                        title='Progresso Diário das Vendas')

    # Gráfico de pizza do progresso anual
    progresso_acumulado = df['Vendas'].sum()
    progresso_percentual = progresso_acumulado / meta_vendas * 100
    fig_anual = px.pie(values=[progresso_percentual, 100 - progresso_percentual],
                       names=['Progresso', 'Restante'],
                       title='Progresso Anual para Atingir a Meta de Vendas')

    return fig_diario, fig_anual

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>